In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from scipy import stats
from sklearn.pipeline import Pipeline

## 시계열 데이터 전처리 사례로 Soft sensor 데이터 일부를 활용했습니다.

In [2]:
data1 = pd.read_csv('soft_sensor.csv')

In [3]:
data1

,Unnamed: 0,CDU2YP12,CDU2YPC3001,CDU2YPC3001_M,CDU2YPC3001_OP,CDU2YPC3001_SP,CDU2YV3AL,FLU_CDU2,REFCDU2_KG_FL,REFCDU2_NM3_FL,CDU2KEROFL
0,2014-01-01,-0.002733,3.553492,0.0,0.0,3.7,-2.014671,0.000000,0.000000,0.000000,50.75
1,2014-01-02,-0.002921,3.519083,0.0,0.0,3.7,-2.046204,0.000000,0.000000,0.000000,51.00
2,2014-01-03,-0.003025,3.530396,0.0,0.0,3.7,-1.995892,0.000000,0.000000,0.000000,51.00
3,2014-01-04,-0.002946,3.518329,0.0,0.0,3.7,-1.914775,0.000000,0.000000,0.000000,54.00
4,2014-01-05,-0.003096,3.535379,0.0,0.0,3.7,-1.907779,0.000000,0.000000,0.000000,54.50
...,...,...,...,...,...,...,...,...,...,...,...
997,2016-11-07,-0.000467,3.589683,0.0,100.3,3.2,0.170158,0.000000,0.000000,0.000000,41.00
998,2016-11-08,-0.000833,3.587742,0.0,100.3,3.2,0.225746,0.000000,0.000000,0.000000,42.50
999,2016-11-09,-0.001050,3.567704,0.0,100.3,3.2,0.463913,0.034692,0.034692,0.027771,40.25
1000,2016-11-15,0.001675,3.815404,0.0,100.3,3.7,0.145146,232.130829,231.742575,133.150246,45.00


In [4]:
datetime_col = 'Unnamed: 0'

## 시계열 데이터를 처리하는 방법을 크게 두가지로 구현하였습니다. 첫번째 방법은 datetime의 각 요소(년월일시분초...)를 timestamp나 각각의 컬럼으로 categorization하는 방법입니다.

In [5]:
from datetime import datetime
import time
import dateutil.parser

class AutoinsightDatetime64Converter(BaseEstimator, TransformerMixin):
    def __init__(self, feature_name, datetime_format=None, populate_features=False, convert_timestamp=False):
        self.feature_name = feature_name
        self.datetime_format = datetime_format
        self.populate_features = populate_features
        self.convert_timestamp = convert_timestamp

    def fit(self, X, y=0, **fit_params):
        return self

    def transform(self, X, y=0):
        cn = self.feature_name
        target_column = X.loc[:, cn]
        # X.loc[:, col] = pd.to_datetime(converting_col, format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')
        if self.datetime_format:
            X.loc[:, cn] = pd.to_datetime(
                target_column,
                format=self.datetime_format,
                errors='coerce'
            )
        else:
            def _parse(x):
                try:
                    # TODO: 매번 format을 유추하기 때문에 느리다;
                    ret = dateutil.parser.parse(x)
                except dateutil.parser.ParserError:
                    ret = datetime.fromtimestamp(0)
                return ret
            X.loc[:, cn] = target_column.map(lambda x: _parse(x))
        if self.populate_features:
            idx = X.columns.get_loc(cn)
            X.insert(idx + 1, cn + '_year', X[cn].dt.year)
            X.insert(idx + 2, cn + '_month', X[cn].dt.month)
            X.insert(idx + 3, cn + '_week', X[cn].dt.week)
            X.insert(idx + 4, cn + '_day', X[cn].dt.day)
            X.insert(idx + 5, cn + '_hour', X[cn].dt.hour)
            X.insert(idx + 6, cn + '_minute', X[cn].dt.minute)
            X.insert(idx + 7, cn + '_dayofweek', X[cn].dt.dayofweek)

        if self.convert_timestamp:
            X.loc[:, cn] = X.loc[:, cn].map(lambda x: time.mktime(
                x.timetuple()
            ))

        return X
    
    
steps = []
steps.append(['datetime_convert', AutoinsightDatetime64Converter(datetime_col, populate_features=True)])

In [6]:
converted = Pipeline(steps).fit_transform(data1)
converted

,Unnamed: 0,Unnamed: 0_year,Unnamed: 0_month,Unnamed: 0_week,Unnamed: 0_day,Unnamed: 0_hour,Unnamed: 0_minute,Unnamed: 0_dayofweek,CDU2YP12,CDU2YPC3001,CDU2YPC3001_M,CDU2YPC3001_OP,CDU2YPC3001_SP,CDU2YV3AL,FLU_CDU2,REFCDU2_KG_FL,REFCDU2_NM3_FL,CDU2KEROFL
0,2014-01-01,2014,1,1,1,0,0,2,-0.002733,3.553492,0.0,0.0,3.7,-2.014671,0.000000,0.000000,0.000000,50.75
1,2014-01-02,2014,1,1,2,0,0,3,-0.002921,3.519083,0.0,0.0,3.7,-2.046204,0.000000,0.000000,0.000000,51.00
2,2014-01-03,2014,1,1,3,0,0,4,-0.003025,3.530396,0.0,0.0,3.7,-1.995892,0.000000,0.000000,0.000000,51.00
3,2014-01-04,2014,1,1,4,0,0,5,-0.002946,3.518329,0.0,0.0,3.7,-1.914775,0.000000,0.000000,0.000000,54.00
4,2014-01-05,2014,1,1,5,0,0,6,-0.003096,3.535379,0.0,0.0,3.7,-1.907779,0.000000,0.000000,0.000000,54.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,2016-11-07,2016,11,45,7,0,0,0,-0.000467,3.589683,0.0,100.3,3.2,0.170158,0.000000,0.000000,0.000000,41.00
998,2016-11-08,2016,11,45,8,0,0,1,-0.000833,3.587742,0.0,100.3,3.2,0.225746,0.000000,0.000000,0.000000,42.50
999,2016-11-09,2016,11,45,9,0,0,2,-0.001050,3.567704,0.0,100.3,3.2,0.463913,0.034692,0.034692,0.027771,40.25
1000,2016-11-15,2016,11,46,15,0,0,1,0.001675,3.815404,0.0,100.3,3.7,0.145146,232.130829,231.742575,133.150246,45.00


## 두번째 방법으로는 datetime 컬럼 자체를 정리하고 이를 이용해 lag를 추가함으로써 기존 regression 모델을 이용화되, 시계열 분석 접근 방식과 유사하게 모델을 구성하도록 합니다. 해당 방법은 regression에만 적용됩니다.

## 가장 먼저 timeseries data row 간의 간격을 일정하게 맞춥니다. 그 간격은 특별히 지정하지 않으면 데이터의 시계열 샘플 빈도를 자동으로 감지하여, 가장 높은 빈도수의 간격으로 정렬합니다.

In [7]:
def datetime_intervals(X, datetime_col):
    X[datetime_col] = pd.to_datetime(X[datetime_col])
    intervals = (X[datetime_col] - X[datetime_col].shift(1)).dropna()
    if intervals.nunique() == 1:
        fixed_interval = True
    else:
        fixed_interval = False
    return intervals, fixed_interval


class AutoinsightTimeseriesResample(BaseEstimator, TransformerMixin):
    def __init__(self, datetime_col, interval='D', how=None):
        self.datetime_col = datetime_col
        self.interval = interval
        self.how = how

    def fit(self, X, y=0, **fit_params):
        self.intervals, self.fixed_interval = datetime_intervals(X, self.datetime_col)
        return self

    def transform(self, X, y=0, **fit_params):
        if self.fixed_interval:
            pass
        else:
            X.index = pd.to_datetime(X[self.datetime_col])
            self.interval = self.intervals.mode()[0]

            if self.how is None:
                X = X.resample(self.interval).mean()
            else:
                pass
        return X

In [8]:
intervals = datetime_intervals(data1, 'Unnamed: 0')

intervals[0].unique() 
# timeseries data row 간의 간격이 제각각입니다

array([  86400000000000,  691200000000000,  518400000000000,
       2851200000000000], dtype='timedelta64[ns]')

In [9]:
steps = []
steps.append(['timeseries_resample', AutoinsightTimeseriesResample(datetime_col)])

In [10]:
steps1 = Pipeline(steps)
resampled = steps1.fit_transform(data1)

In [11]:
resampled

,Unnamed: 0_year,Unnamed: 0_month,Unnamed: 0_week,Unnamed: 0_day,Unnamed: 0_hour,Unnamed: 0_minute,Unnamed: 0_dayofweek,CDU2YP12,CDU2YPC3001,CDU2YPC3001_M,CDU2YPC3001_OP,CDU2YPC3001_SP,CDU2YV3AL,FLU_CDU2,REFCDU2_KG_FL,REFCDU2_NM3_FL,CDU2KEROFL
Unnamed: 0,,,,,,,,,,,,,,,,,
2014-01-01,2014.0,1.0,1.0,1.0,0.0,0.0,2.0,-0.002733,3.553492,0.0,0.0,3.7,-2.014671,0.000000,0.000000,0.000000,50.75
2014-01-02,2014.0,1.0,1.0,2.0,0.0,0.0,3.0,-0.002921,3.519083,0.0,0.0,3.7,-2.046204,0.000000,0.000000,0.000000,51.00
2014-01-03,2014.0,1.0,1.0,3.0,0.0,0.0,4.0,-0.003025,3.530396,0.0,0.0,3.7,-1.995892,0.000000,0.000000,0.000000,51.00
2014-01-04,2014.0,1.0,1.0,4.0,0.0,0.0,5.0,-0.002946,3.518329,0.0,0.0,3.7,-1.914775,0.000000,0.000000,0.000000,54.00
2014-01-05,2014.0,1.0,1.0,5.0,0.0,0.0,6.0,-0.003096,3.535379,0.0,0.0,3.7,-1.907779,0.000000,0.000000,0.000000,54.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-11-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 일정한 간격으로 timeseries 데이터를 재배치하는 과정에서 원래 그 시간대에 데이터가 없었던 경우에는 결측값이 생길 수 있습니다 (2016-11-12~14). 이 경우 주변 값을 토대로 Interpolate하여 결측값이 없도록 합니다.

In [12]:
class AutoinsightTimeseriesInterpolate(BaseEstimator, TransformerMixin):
    def __init__(self, method='linear'):
        self.method = method

    def fit(self, X, y=0, **fit_params):
        if X.isnull().values.any():
            self.need_interpolation = True
        else:
            self.need_interpolation = False
        return self

    def transform(self, X, y=0, **fit_params):
        if self.need_interpolation:
            X = X.interpolate(method=self.method)
        else:
            pass
        return X

In [13]:
steps.append(['timeseries_interpolate', AutoinsightTimeseriesInterpolate()])
steps2 = Pipeline(steps)
resampled2 = steps2.fit_transform(data1)

In [14]:
resampled2

,Unnamed: 0_year,Unnamed: 0_month,Unnamed: 0_week,Unnamed: 0_day,Unnamed: 0_hour,Unnamed: 0_minute,Unnamed: 0_dayofweek,CDU2YP12,CDU2YPC3001,CDU2YPC3001_M,CDU2YPC3001_OP,CDU2YPC3001_SP,CDU2YV3AL,FLU_CDU2,REFCDU2_KG_FL,REFCDU2_NM3_FL,CDU2KEROFL
Unnamed: 0,,,,,,,,,,,,,,,,,
2014-01-01,2014.0,1.0,1.000000,1.0,0.0,0.0,2.000000,-0.002733,3.553492,0.0,0.0,3.700000,-2.014671,0.000000,0.000000,0.000000,50.750000
2014-01-02,2014.0,1.0,1.000000,2.0,0.0,0.0,3.000000,-0.002921,3.519083,0.0,0.0,3.700000,-2.046204,0.000000,0.000000,0.000000,51.000000
2014-01-03,2014.0,1.0,1.000000,3.0,0.0,0.0,4.000000,-0.003025,3.530396,0.0,0.0,3.700000,-1.995892,0.000000,0.000000,0.000000,51.000000
2014-01-04,2014.0,1.0,1.000000,4.0,0.0,0.0,5.000000,-0.002946,3.518329,0.0,0.0,3.700000,-1.914775,0.000000,0.000000,0.000000,54.000000
2014-01-05,2014.0,1.0,1.000000,5.0,0.0,0.0,6.000000,-0.003096,3.535379,0.0,0.0,3.700000,-1.907779,0.000000,0.000000,0.000000,54.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-11-12,2016.0,11.0,45.500000,12.0,0.0,0.0,1.500000,0.000313,3.691554,0.0,100.3,3.450000,0.304529,116.082760,115.888633,66.589008,42.625000
2016-11-13,2016.0,11.0,45.666667,13.0,0.0,0.0,1.333333,0.000767,3.732837,0.0,100.3,3.533333,0.251401,154.765450,154.506614,88.776088,43.416667
2016-11-14,2016.0,11.0,45.833333,14.0,0.0,0.0,1.166667,0.001221,3.774121,0.0,100.3,3.616667,0.198274,193.448140,193.124594,110.963167,44.208333


## 그리고 시계열 데이터의 autocorrelation을 설명하기 위해 이전 시기의 y값들을 새로운 feature로 추가합니다 (lag). 몇 개의 이전 시기를 lag로 지정할지는 사용자가 int나 int list로 지정할 수도 있고, 'auto'로 지정하면 lag_selector가 데이터 내에서 탐색하도록 할 수도 있습니다.

In [15]:
class AutoinsightTimeseriesMakeLags(BaseEstimator, TransformerMixin):
    def __init__(self, target_col, lag=int(1)):
        self.target_col = target_col
        self.lag = lag

    def fit(self, X, y=0, **fit_params):
        return self

    def transform(self, X, y=0, **fit_params):
        if self.lag == 'auto':
            X = lag_selector(X, self.target_col)
        elif isinstance(self.lag, int):
            X['{}_lag{}'.format(self.target_col, self.lag)] = X[self.target_col].shift(self.lag)
            X = X.dropna(axis=0)
        elif isinstance(self.lag, list):
            for i in self.lag:
                X['{}_lag{}'.format(self.target_col, i)] = X[self.target_col].shift(i)
            X = X.dropna(axis=0)
        else:
            raise TypeError("Wrong type of lag input. It should be an integer or a list of integers")
        return X


def lag_selector(X, target_col, lags=None):
    if lags is None:
        lags = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # a list of integers
    for i in lags:
        X['{}_lag{}'.format(target_col, i)] = X[target_col].shift(i)
        cor = stats.pearsonr(X[target_col][i:], X['{}_lag{}'.format(target_col, i)][i:])
        if cor[1] > 0.05:
            X.drop('{}_lag{}'.format(target_col, i), axis=1)
    return X

In [16]:
steps.append(['timeseries_lags', AutoinsightTimeseriesMakeLags('CDU2KEROFL', lag=[1,2,3])])
steps3 = Pipeline(steps)
resampled3 = steps3.fit_transform(data1)

In [17]:
resampled3

,Unnamed: 0_year,Unnamed: 0_month,Unnamed: 0_week,Unnamed: 0_day,Unnamed: 0_hour,Unnamed: 0_minute,Unnamed: 0_dayofweek,CDU2YP12,CDU2YPC3001,CDU2YPC3001_M,CDU2YPC3001_OP,CDU2YPC3001_SP,CDU2YV3AL,FLU_CDU2,REFCDU2_KG_FL,REFCDU2_NM3_FL,CDU2KEROFL,CDU2KEROFL_lag1,CDU2KEROFL_lag2,CDU2KEROFL_lag3
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
2014-01-04,2014.0,1.0,1.000000,4.0,0.0,0.0,5.000000,-0.002946,3.518329,0.0,0.0,3.700000,-1.914775,0.000000,0.000000,0.000000,54.000000,51.000000,51.000000,50.750000
2014-01-05,2014.0,1.0,1.000000,5.0,0.0,0.0,6.000000,-0.003096,3.535379,0.0,0.0,3.700000,-1.907779,0.000000,0.000000,0.000000,54.500000,54.000000,51.000000,51.000000
2014-01-06,2014.0,1.0,2.000000,6.0,0.0,0.0,0.000000,-0.002962,3.538012,0.0,0.0,3.700000,-1.870142,0.000000,0.000000,0.000000,54.500000,54.500000,54.000000,51.000000
2014-01-07,2014.0,1.0,2.000000,7.0,0.0,0.0,1.000000,-0.002933,3.547592,0.0,0.0,3.700000,-1.935725,0.000000,0.000000,0.000000,53.750000,54.500000,54.500000,54.000000
2014-01-08,2014.0,1.0,2.000000,8.0,0.0,0.0,2.000000,-0.003021,3.552079,0.0,0.0,3.700000,-1.867983,0.000000,0.000000,0.000000,64.500000,53.750000,54.500000,54.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-11-12,2016.0,11.0,45.500000,12.0,0.0,0.0,1.500000,0.000313,3.691554,0.0,100.3,3.450000,0.304529,116.082760,115.888633,66.589008,42.625000,41.833333,41.041667,40.250000
2016-11-13,2016.0,11.0,45.666667,13.0,0.0,0.0,1.333333,0.000767,3.732837,0.0,100.3,3.533333,0.251401,154.765450,154.506614,88.776088,43.416667,42.625000,41.833333,41.041667
2016-11-14,2016.0,11.0,45.833333,14.0,0.0,0.0,1.166667,0.001221,3.774121,0.0,100.3,3.616667,0.198274,193.448140,193.124594,110.963167,44.208333,43.416667,42.625000,41.833333
